# metacognitive prompting

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from dotenv import load_dotenv
import sys
import queue

load_dotenv()

In [ ]:
sys.path.append(os.getenv("SYS_PATH"))
print(f'{os.getenv("SYS_PATH")}')
print(sys.path)

In [ ]:
import openai
from openai import OpenAI, OpenAIError
from ipywidgets import widgets
from IPython.display import display, clear_output
from bson.objectid import ObjectId
from typing import Optional, List, Dict
import threading

from models.user import create_user
from models.chat_message import create_chat_message, ChatMessageData
from models.conversation import create_conversation
from models.chatbot_response import create_chatbot_response
from models.expectation import create_expectation
from models.expectation_revision import create_expectation_revision
from models.violation import create_violation

from utils.model_operations import create_model, get_model, get_chat_history
from services.mock_service import mock_user_data, mock_expectation_data, mock_expectation_revision_data, mock_message_data, mock_violation_data, mock_chatbot_response_data
from services.prompt_builder_service import build_context_prompt, build_expectation_prompt, build_expectation_revision_prompt, build_voe_prompt
from utils.helpers import CustomOutput

In [ ]:
%run ../services/mongo_service.py

In [ ]:
from services.mongo_service import mongo_client, ping_client
db_client = mongo_client()
ping_client(db_client)

In [ ]:
text_input = ""
chat_output = CustomOutput()
user_input_queue = queue.Queue()
expectations_queue = queue.Queue()

saved_user_id = None
saved_conversation_id = None
set_user = None
set_conversation = None
chat_history = []

In [ ]:
def test_items():
    global saved_user_id, saved_conversation_id
    chat_output.append_stdout(f"""Here follow some examples of:
    - user
    - conversation
    - expectation
    - expectation revision
    - user chat message
    - violation (VoE thought & context)
    - chatbot response message""")
    test_user = create_user(email=mock_user_data["email"], password=mock_user_data["password"])
    saved_user_id = create_model(collection_name="users", model_data=test_user, client=db_client)
    chat_output.append_stdout(f"test_user: {test_user} with id: {saved_user_id}\nis of type{type(test_user)}\n\n", stream=True)
    
    test_conversation = create_conversation(user_id=saved_user_id)
    saved_conversation_id = create_model(collection_name="conversations", model_data=test_conversation, client=db_client)
    chat_output.append_stdout(f"test_conversation: {test_conversation} with id: {saved_conversation_id}\nis of type{type(test_conversation)}\n\n", stream=True)
    
    test_expectation = create_expectation(reasoning=mock_expectation_data["reasoning"], user_predictions=mock_expectation_data["user_predictions"],     additional_data=mock_expectation_data["additional_data"])
    saved_expectation_id = create_model(collection_name="expectations", model_data=test_expectation, client=db_client)
    chat_output.append_stdout(f"test_expectation: {test_expectation} with id: {saved_expectation_id}\nis of type{type(test_expectation)}\n\n", stream=True)
    
    test_expectation_revision = create_expectation_revision(revised_input_possibilities=mock_expectation_revision_data["revised_input_possibilities"],     prediction_error=-0.045, initial_expectation_id=saved_expectation_id)
    saved_expectation_revision_id = create_model(collection_name="expectation_revisions", model_data=test_expectation_revision, client=db_client)
    chat_output.append_stdout(f"test_expectation_revision: {test_expectation_revision} with id: {saved_expectation_revision_id}\nis of type{type    (test_expectation_revision)}\n\n", stream=True)
    
    test_message = create_chat_message(user_id=saved_user_id, content=mock_message_data["content"], conversation_id=saved_conversation_id)
    saved_message_id = create_model(collection_name="chat_messages", model_data=test_message, client=db_client)
    chat_output.append_stdout(f"test_message: {test_message} with id: {saved_message_id}\nis of type{type(saved_message_id)}\n\n", stream=True)
    
    test_violation = create_violation(last_llm_response_id=None, expectation_id=saved_expectation_id, voe_thought=mock_violation_data["voe_thought"])
    saved_violation_id = create_model(collection_name="violations", model_data=test_violation, client=db_client)
    chat_output.append_stdout(f"test_violation: {test_violation} with id: {saved_violation_id}\nis of type{type(test_violation)}\n\n", stream=True)
    
    test_chatbot_response = create_chatbot_response(content=mock_chatbot_response_data['content'], conversation_id=saved_conversation_id, response_to_id=saved_message_id)
    saved_response_id = create_model(collection_name="chatbot_responses", model_data=test_chatbot_response, client=db_client)
    chat_output.append_stdout(f"test_chatbot_response {test_chatbot_response} with id: {saved_response_id}\nis of type{type(test_chatbot_response)}")

    get_model(collection_name="users", model_id=saved_user_id, client=db_client)


In [ ]:
apikey = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(
    api_key=apikey
)
openai.api_key=apikey
print(openai_client)

In [ ]:
def create_thought(prompts: List[Dict[str, str]]) -> str:
    try:
        
        thought_completion = openai_client.chat.completions.create(
            messages=prompts,
            model="gpt-3.5-turbo"
        )
        print(thought_completion)
        
        thought = thought_completion.choices[0].message.content
        
        return str(thought)
    except OpenAIError as e:
        #Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")
        raise e
        

In [ ]:
def stream_chatbot() -> str:
    try:
        chat_completion_stream = openai_client.chat.completions.create(
            messages=[
                {
                    "role": "user" if item.get("user_id") else "assistant",
                    "content": item["content"]
                }
                for item in chat_history
            ],
            model="gpt-3.5-turbo",
            stream=True
        )
        print(chat_completion_stream)
        message = ""
        chat_output.append_stdout(f"Chatbot: ", stream=True)
        for chunk in chat_completion_stream:
            if chunk.choices[0].delta.content is not None:
                chat_output.append_stdout(f"{chunk.choices[0].delta.content}", stream=True)
                message += chunk.choices[0].delta.content
        chat_output.append_stdout(f"\n")
        
        return message
    except OpenAIError as e:
        #Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")
        raise e

In [ ]:
def print_user_message(user_message):
    chat_output.append_stdout(f"You: {user_message}")

In [ ]:
def handle_button_click():
    chat_output.append_stdout(f"handle button clicked", debug=True)
    chat_output.append_stdout(f"text_input.value: {text_input.value}", debug=True)
    user_input_queue.put(text_input.value)
    text_input.value=''
    chat_output.append_stdout(f"handle button clicked after setting queue value", debug=True)
        

In [ ]:
def initialize_chat(user_id: ObjectId, conversation_id: Optional[ObjectId] = None):
    global set_user, set_conversation, chat_history
    set_user = get_model(collection_name="users", model_id=saved_user_id, client=db_client)
    if not conversation_id:
        new_conversation = create_conversation(user_id=user_id)
        conversation_id = create_model(collection_name="conversations", model_data=new_conversation, client=db_client)
    set_conversation = get_model(collection_name="conversations", model_id=conversation_id, client=db_client)
    
    chat_history = get_chat_history(conversation_id=conversation_id, client=db_client)
    


In [ ]:
def print_chat_history():
    chat_output.append_stdout(f"The chat history that belongs to this user and conversation:")
    for item in chat_history:
        if item.get('user_id'):
            print_user_message(item.get('content'))
        else:
            chat_output.append_stdout(f"Chatbot: ", stream=True)
            chat_output.append_stdout(item.get('content'))


In [ ]:
def user_prediction_task() -> dict:
    # invoke reasoning process with Engagement Monitor Service
    reasoning = mock_expectation_data["reasoning"]
    
    # invoke user predictions with LLM Service
    # user_predictions = mock_expectation_data["user_predictions"]
    expectation_prompt = build_expectation_prompt(first_message=True if len(chat_history) == 0 else False)
    prompt = [
        {"role": "system", "content": expectation_prompt}
    ]
    user_predictions = create_thought(prompts=prompt)
    
    
    # invoke vector db fact fetching with Knowledge Assessment Service
    additional_data = mock_expectation_data["additional_data"]
    
    # create and save Expectation
    new_expectation = create_expectation(reasoning=reasoning, user_predictions=user_predictions, additional_data=additional_data)
    expectation_id = create_model(collection_name="expectations", model_data=new_expectation, client=db_client)
    
    # invoke prediction improvement with LLM Service
    # improved_predictions = mock_expectation_revision_data["revised_input_possibilities"]
    expectation_revision_prompt = build_expectation_revision_prompt(user_facts=additional_data)
    prompt = [
        {"role": "system", "content": expectation_revision_prompt}
    ]
    improved_predictions  = create_thought(prompts=prompt)
    expectations_queue.put(improved_predictions)
    # invoke prediction error calculation with ??? Service
    prediction_error = 0.0
    
    # create and save ExpectationRevision
    new_expectation_revision = create_expectation_revision(revised_input_possibilities=improved_predictions, prediction_error=prediction_error, initial_expectation_id=expectation_id)
    expectation_revision_id = create_model(collection_name="expectation_revisions", model_data=new_expectation_revision, client=db_client)
    
    expectation_revision_dict = get_model(collection_name="expectation_revisions", model_id=expectation_revision_id, client=db_client)
    
    chat_output.append_stdout(f"USER PREDICTION TASK COMPLETED\nexpectation_revision_dict ID: {expectation_revision_dict['_id']}", debug=True)
    
    return expectation_revision_dict
    

In [ ]:
def user_input_task() -> dict:
    chat_output.append_stdout(f"USER INPUT TASK", debug=True)
    chat_output.append_stdout(f"user_input_queue: {user_input_queue.__dict__}", debug=True)
    
    try:
        user_input = user_input_queue.get()
        
        print_user_message(user_input)
        
        new_chat_message = create_chat_message(user_id=set_user['_id'], content=user_input, conversation_id=set_conversation['_id'])
        chat_message_id = create_model(collection_name="chat_messages", model_data=new_chat_message, client=db_client)
        chat_message_dict = get_model(collection_name="chat_messages", model_id=chat_message_id, client=db_client)
        
        chat_output.append_stdout(f"USER INPUT TASK COMPLETED\nchat message dict: {chat_message_dict}", debug=True)
        
        chat_history.append(chat_message_dict)
        
        return chat_message_dict
    
    except Exception as e:
        raise e

In [ ]:
def voe_task(expectation_id: ObjectId) -> dict:
    try:
        # invoke violation of expectation with LLM service
        # voe_thought = mock_violation_data['voe_thought']
        last_expectation = expectations_queue.get()
        last_llm_response = chat_history[len(chat_history)-2].get('content')
        last_input = chat_history[len(chat_history)-1].get('content')

        voe_prompt = build_voe_prompt(last_llm_response=last_llm_response, expectation=last_expectation, violation=last_input)
        prompt = [
            {"role": "system", "content": voe_prompt},
        ]
        voe_thought = create_thought(prompt)
        
        # implement DB call
        # last_llm_response = None
        
        new_violation = create_violation(last_llm_response_id=last_llm_response, expectation_id=expectation_id, voe_thought=voe_thought)
        violation_id = create_model(collection_name="violations", model_data=new_violation, client=db_client)
        violation_dict = get_model(collection_name="violations", model_id=violation_id, client=db_client)
        
        return violation_dict
    
    except Exception as e:
        raise e

In [ ]:
def chatbot_response_task(response_to_id: ObjectId):
    try:
        user_message_dict = get_model(collection_name="chat_messages", model_id=response_to_id, client=db_client)
        chat_output.append_stdout(f"{user_message_dict}", debug=True)
        user_message_model = user_message_dict
        chat_output.append_stdout(f"user_message_model: {user_message_model}",debug=True)
        
        # invoke prompt creation with useful information
        
        # invoke chatbot response with LLM Service
        generated_response = stream_chatbot()
        chat_output.append_stdout(f"generated_response: {generated_response}", debug=True)
        
        # save final API output as "content"
        content = generated_response
        
        new_chatbot_response = create_chatbot_response(content=content, conversation_id=set_conversation['_id'], response_to_id=response_to_id)
        response_id = create_model(collection_name="chatbot_responses", model_data=new_chatbot_response, client=db_client)
        response_dict = get_model(collection_name="chatbot_responses", model_id=response_id, client=db_client)

        chat_history.append(response_dict)
        
        return response_dict
    
    except Exception as e:
        raise e

In [ ]:
def conversation_loop():
    global text_input
    active = True
    text_input = widgets.Text(placeholder='Type your message here...')
    submit_button = widgets.Button(description='Submit')
    exit_button = widgets.Button(description='Exit')

    def on_submit_button_clicked(b):
        with chat_output:
            handle_button_click()
            if active:
                # invoke user prediction task and return prediction object
                expectation_dict = user_prediction_task()
                chat_output.append_stdout(f"Received prediction: {expectation_dict}", debug=True)
                expectation_id = expectation_dict["_id"]

                # invoke user input task and return user message
                message_dict = user_input_task()
                chat_output.append_stdout(f"Received message_dict: {message_dict}", debug=True)                
                message_id = message_dict["_id"]
    
                # invoke violation of expectation task and return violation object
                violation_dict = voe_task(expectation_id=expectation_id)
                chat_output.append_stdout(f"Received violation: {violation_dict}", debug=True)
                
                # invoke vector db fact storing with Knowledge Assessment Service
                
                # invoke chatbot response task and return chatbot message
                chatbot_response_dict = chatbot_response_task(message_id)
                chat_output.append_stdout(f"Received response: {chatbot_response_dict}", debug=True)

    submit_button.on_click(on_submit_button_clicked)

    def on_exit_button_clicked(b):
        nonlocal active
        with chat_output:
            active = False
            chat_output.clear_output()
            chat_output.append_stdout("Exiting conversation loop...")

    exit_button.on_click(on_exit_button_clicked)

    display(widgets.VBox([chat_output, text_input, submit_button, exit_button]))

  

In [ ]:
def main():
    test_items()
    initialize_chat(saved_user_id, saved_conversation_id)
    chat_output.append_stdout(f"The user and conversation objects this conversation is set to:")
    chat_output.append_stdout(f"user: {set_user}\nconversation: {set_conversation}")
    print_chat_history()
    conversation_loop()

if __name__ == "__main__": main()